[爬蟲案例分享]
- 難點：

    1.資料量大，但每個查詢最多150個分頁，因此需要將查詢的範圍限縮。
    解決方式是在查詢時組合地區與職務類型，進行細緻的查詢。

    2.雖然幾乎沒有設定反爬蟲的機制，但每個 request 送的速度太快仍然會有失敗的狀況發生
    用while迴圈，當request成功是繼續前往下一頁爬蟲，request成功但資料小於20筆，查詢下一個組合，request等於20，表示還有資料需要繼續抓下一個分頁的資料。

    3.因為資料量大，建議按照組別存一次資料，這樣遇到錯誤時才不用重新爬資料。

    
- 因為查詢職缺最多有20\*150=3000筆資料，所以要將查詢的的條件透過條件進行篩選、排列組合，藉此撈出完整的資料
- 由公司資訊：地址，聯絡電話、職務
- 縣市和職務的分類有個資的代號，但中間有經過變(如縣市合併)更並不是單純的序列，因此要花時間找一下
- 可以看各個縣市的職缺數量與職務類型，進一步分析需要什麼技能
- 你要在哪裡，需要什麼技能、待遇如何
- 可以去串各縣市的人口數、教育程度等資訊
- 縣市的產業結構
- 可行的分析項目

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
from IPython.display import clear_output

In [ ]:
headers={'user-agents':'GoogleBot'}

In [2]:
# 地區代碼
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url)
df1 = []
for i in resp.json()[0]['n']:
    ndf = pd.DataFrame(i['n'])
    ndf['city'] = i['des']
    df1.append(ndf)
df1=pd.concat(df1, ignore_index=True)
df1 = df1.loc[:,['city','des','no']]
df1 = df1.sort_values('no')
df1

,city,des,no
0,台北市,台北市中正區,6001001001
1,台北市,台北市大同區,6001001002
2,台北市,台北市中山區,6001001003
3,台北市,台北市松山區,6001001004
4,台北市,台北市大安區,6001001005
...,...,...,...
360,金門縣,金門縣烏坵鄉,6001022006
361,連江縣,連江縣南竿鄉,6001023001
362,連江縣,連江縣北竿鄉,6001023002
363,連江縣,連江縣莒光鄉,6001023003


In [3]:
url= 'https://static.104.com.tw/category-tool/json/JobCat.json'
resp = requests.get(url)
df2 = []
for i in resp.json():
    for j in i['n']:
        ndf = pd.DataFrame(j['n'])
        ndf['des1'] = i['des']# 職務大分類
        ndf['des2'] = j['des']# 職務小分類
        df2.append(ndf)
df2 = pd.concat(df2, ignore_index=True)
df2 = df2.loc[:,['des1', 'des2', 'des', 'no']]
df2 = df2.sort_values('no')
df2

,des1,des2,des,no
1,經營╱人資類,經營╱幕僚類人員,經營管理主管,2001001001
0,經營╱人資類,經營╱幕僚類人員,儲備幹部,2001001002
2,經營╱人資類,經營╱幕僚類人員,主管特別助理,2001001003
3,經營╱人資類,人力資源類人員,人力資源主管,2001002001
6,經營╱人資類,人力資源類人員,人力資源人員,2001002002
...,...,...,...,...
461,其他職類,其他類人員,加油員,2018002010
462,其他職類,其他類人員,派報生╱傳單派送,2018002011
463,其他職類,其他類人員,清潔工╱資源回收人員,2018002012
464,其他職類,其他類人員,家事服務人員,2018002013


In [4]:
tmp = pd.DataFrame([re.sub('\.pkl','',file)for file in os.listdir('./data')],columns=['no'])
df1 = pd.merge(df1, tmp, how='left',on='no',indicator=True)
df1 = df1.loc[df1['_merge']!='both',:]
df1

,city,des,no,_merge
252,高雄市,高雄市鹽埕區,6001016004,left_only
253,高雄市,高雄市鼓山區,6001016005,left_only
254,高雄市,高雄市旗津區,6001016006,left_only
255,高雄市,高雄市前鎮區,6001016007,left_only
256,高雄市,高雄市三民區,6001016008,left_only
...,...,...,...,...
360,金門縣,金門縣烏坵鄉,6001022006,left_only
361,連江縣,連江縣南竿鄉,6001023001,left_only
362,連江縣,連江縣北竿鄉,6001023002,left_only
363,連江縣,連江縣莒光鄉,6001023003,left_only


In [5]:
columns = ['公司名稱','公司編號','公司類別','公司類別描述', '公司連結','職缺名稱','職務性質','職缺大分類', '職缺中分類','職缺小分類',
           '職缺編號', '職務內容','更新日期', '職缺連結', '標籤','公司地址','地區','經歷','學歷']

for areades, areacode in zip(df1['des'],df1['no']):
    values = []
    for jobdes1, jobdes2, jobdes, jobcode in zip(df2['des1'], df2['des2'], df2['des'], df2['no']):
        print(areades, ' | ', jobdes1, ' - ', jobdes2, ' - ' ,jobdes)
        page = 1
        while page <150:
            try:
                url = 'https://www.104.com.tw/jobs/search/?ro=0&jobcat={}&jobcatExpansionType=1&area={}&order=11&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(jobcode, areacode, page)
                print(url)
                resp = requests.get(url,headers=headers)
                soup = BeautifulSoup(resp.text)
                soup2 = soup.find('div',{'id':'js-job-content'}).findAll('article',{'class':'b-block--top-bord job-list-item b-clearfix js-job-item'})
                print(len(soup2))

                for job in soup2:
                                        
                    update_date = job.find('span',{'class':'b-tit__date'}).text
                    update_date = re.sub('\r|\n| ','',update_date)

                    try:
                        address = job.select('ul > li > a')[0]['title']
                        address = re.findall('公司住址：(.*?)$',address)[0]
                    except:
                        address = ''
                   
                    loc = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[0].text
                    exp = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[1].text
                    try:
                        edu = job.find('ul',{'class':'b-list-inline b-clearfix job-list-intro b-content'}).findAll('li')[2].text
                    except:
                        edu = ''
                    
                    try:
                        content = job.find('p').text
                    except:
                        content = ''
                    try:
                        tags = [tag.text for tag in soup2[0].find('div',{'class':'job-list-tag b-content'}).findAll('span')]
                    except:
                        tags = []
                    
                    
                    value = [job['data-cust-name'], # 公司名稱
                             job['data-cust-no'], # 公司編號
                             job['data-indcat'], # 公司類別
                             job['data-indcat-desc'], # 公司類別描述
                             job.select('ul > li > a')[0]['href'], # 公司連結
                             job['data-job-name'],# 職缺名稱
                             job['data-job-ro'], # 職務性質 _判斷全職兼職 1全職/2兼職/3高階/4派遣/5接案/6家教
                             jobdes1, # 職缺大分類
                             jobdes2, # 職缺中分類
                             jobdes, # 職缺小分類
                             job['data-job-no'],# 職缺編號
                             content, # 職務內容
                             update_date, # 更新日期
                             job.find('a',{'class':'js-job-link'})['href'], # 職缺連結
                             tags, # 標籤
                             address,# 公司地址
                             loc, # 地區
                             exp,# 經歷
                             edu  # 學歷
                            ]
                    values.append(value)
                
                page+=1
                print(len(values))
                if len(soup2) < 20:
                    break
            except:
                print('Retry')
        
    df = pd.DataFrame()
    df = pd.DataFrame(values, columns=columns)
    df.to_pickle('./data/' + areacode + '.pkl')
    clear_output()
    print('===================================  Save Data  ===================================')

===================================  Save Data  ===================================


In [10]:
df = []
for i in os.listdir('./data/'):
    ndf = pd.read_pickle('./data/' + i)
    df.append(ndf)
df = pd.concat(df, ignore_index=True)
df

,公司名稱,公司編號,公司類別,公司類別描述,公司連結,職缺名稱,職務性質,職缺大分類,職缺中分類,職缺小分類,職缺編號,職務內容,更新日期,職缺連結,標籤,公司地址,地區,經歷,學歷
0,德信綜合證券股份有限公司,23470432000,1004002002,證券及期貨業,//www.104.com.tw/company/as5ot8g?jobsource=201...,董事長室特助,1,經營╱人資類,經營╱幕僚類人員,經營管理主管,11085193,1.提供董事長經營管理建議及執行\r\n2.協助處理法律相關案件\r\n,6/18,//www.104.com.tw/job/6lle1?jobsource=2018indexpoc,"[待遇面議, 員工260人]",台北市中正區新生南路一段50號3樓,台北市中正區,2年以上,大學
1,聯邦保全股份有限公司,70691148000,1009006001,保全樓管相關業,//www.104.com.tw/company/wh3omw0?jobsource=201...,綜管勤務幹部,1,經營╱人資類,經營╱幕僚類人員,經營管理主管,11303945,1. 保全人員班表排定製作(每月底前)\r\n2. 保全勤務調度(臨時或緊急狀況排除)\r\...,6/18,//www.104.com.tw/job/6qa6h?jobsource=2018indexpoc,"[待遇面議, 員工260人]",台北市中正區羅斯福路四段162號12樓,台北市中正區,2年以上,高中
2,恬媞琳美容坊,13849529000,1009005002,美容／美體業,//www.104.com.tw/company/6d1ngyw?jobsource=201...,門市店長(台大店),1,經營╱人資類,經營╱幕僚類人員,經營管理主管,8368962,1.美容店務管理。\r\n2.人員在職訓練管理。\r\n3.客戶售後服務之規劃安排\r\n4...,6/18,//www.104.com.tw/job/4zdj6?jobsource=2018indexpoc,"[待遇面議, 員工260人]",台北市中正區羅斯福路四段50號2樓,台北市中正區,2年以上,學歷不拘
3,財團法人信誼基金會,82003814000,1013002002,職業團體,//www.104.com.tw/company/11o6ybu8?jobsource=20...,數位長,1,經營╱人資類,經營╱幕僚類人員,經營管理主管,10622980,1. 整合組織資源，帶領組織數位轉型。\r\n2. 資料庫與跨平台服務之建構.營運及推廣。\r\n,6/17,//www.104.com.tw/job/6boqs?jobsource=2018indexpoc,"[待遇面議, 員工260人]",台北市中正區重慶南路二段75號,台北市中正區,5年以上,大學
4,財團法人綠色和平基金會,26324671000,1013001003,國際組織及外國機構,//www.104.com.tw/company/c3d1254?jobsource=201...,Action Unit Manager 行動統籌經理,1,經營╱人資類,經營╱幕僚類人員,經營管理主管,11673371,We are HIRING!!!\r\n\r\nGreenpeace is a non-pr...,6/17,//www.104.com.tw/job/6y78b?jobsource=2018indexpoc,"[待遇面議, 員工260人]",台北市中正區重慶南路一段109號,台北市中正區,5年以上,大學
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551659,佑呈開發有限公司,130000000136697,1012002001,環境衛生及污染防治服務業,//www.104.com.tw/company/1a2x6bkws9?jobsource=...,【環保工程】約聘專案工程師（連江縣）,1,行銷╱企劃╱專案管理類,專案╱產品管理類人員,其他專案管理師,11616392,"協助公司連江縣北竿鄉專案計畫執行與管理,包含: \r\n1、環保專案計畫執行溝通協調 \r\...",6/08,//www.104.com.tw/job/6wz9k?jobsource=2018indexpoc,"[月薪28,000~49,000元]",連江縣北竿鄉,連江縣北竿鄉,經歷不拘,專科
551660,佑呈開發有限公司,130000000136697,1012002001,環境衛生及污染防治服務業,//www.104.com.tw/company/1a2x6bkws9?jobsource=...,【環保工程】約聘專案工程師（連江縣）,1,生產製造╱品管╱環衛類,環境安全衛生類人員,環境工程人員 / 工程師,11616392,"協助公司連江縣北竿鄉專案計畫執行與管理,包含: \r\n1、環保專案計畫執行溝通協調 \r\...",6/08,//www.104.com.tw/job/6wz9k?jobsource=2018indexpoc,"[月薪28,000~49,000元]",連江縣北竿鄉,連江縣北竿鄉,經歷不拘,專科
551661,財團法人中華民國唐氏症基金會,81598577000,1013003001,社會福利服務業,//www.104.com.tw/company/11hhooyw?jobsource=20...,托育人員(連江縣西莒社區公共托育家園),1,學術╱教育╱輔導類,教育輔導類人員,教保員,11544859,嬰幼兒照護與保育工作(供住宿),6/15,//www.104.com.tw/job/6vg2j?jobsource=2018indexpoc,"[月薪35,000~38,000元, 員工200人]",連江縣莒光鄉青帆村18號,連江縣莒光鄉,經歷不拘,高中
551662,財團法人中華民國唐氏症基金會,81598577000,1013003001,社會福利服務業,//www.104.com.tw/company/11hhooyw?jobsource=20...,托育人員(連江縣東莒社區公共托育家園),1,學術╱教育╱輔導類,教育輔導類人員,教保員,11544841,嬰幼兒照護與保育工作(供住宿),6/15,//www.104.com.tw/job/6vg21?jobsource=2018indexpoc,"[月薪35,000~38,000元, 員工200人]",連江縣莒光鄉大坪村4號,連江縣莒光鄉,經歷不拘,高中


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551664 entries, 0 to 551663
Data columns (total 19 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   公司名稱    551664 non-null  object
 1   公司編號    551664 non-null  object
 2   公司類別    551664 non-null  object
 3   公司類別描述  551664 non-null  object
 4   公司連結    551664 non-null  object
 5   職缺名稱    551664 non-null  object
 6   職務性質    551664 non-null  object
 7   職缺大分類   551664 non-null  object
 8   職缺中分類   551664 non-null  object
 9   職缺小分類   551664 non-null  object
 10  職缺編號    551664 non-null  object
 11  職務內容    551664 non-null  object
 12  更新日期    551664 non-null  object
 13  職缺連結    551664 non-null  object
 14  標籤      551664 non-null  object
 15  公司地址    551664 non-null  object
 16  地區      551664 non-null  object
 17  經歷      551664 non-null  object
 18  學歷      551664 non-null  object
dtypes: object(19)
memory usage: 80.0+ MB


In [11]:
df.to_excel('./JobList.xlsx')

In [16]:
soup2[16].select('ul > li > a')

[<a href="//case.104.com.tw" target="_blank" title="公司名：104外包網">
                         104外包網
                     </a>]

In [15]:
job.select('ul > li')[4]

IndexError: list index out of range